# Engenharia de Dados - Camada Gold (Modelagem Star Schema)

**Objetivo:**

Criar tabelas modeladas para consumo analítico (BI), a metodologia Kimball (Star Schema).

**Estrutura do Modelo:**

* **Dimensões (`dim_`):** Clientes, Produtos, Vendedores, Pedidos, Calendário.

* **Fatos (`fct_`):** Vendas (Granularidade: Item do pedido) e Pagamentos (Rastreabilidade/Foco Financeiro)

**Regras de Negócio:** 

1. Enriquecer a dimensão de produtos com a tradução da categoria.

2. Trazer a análise de sentimento da IA para a tabela fato de vendas.

3. Calcular métricas de prazo de entrega (Lead Time).

In [0]:
from pyspark.sql.functions import col, lit, explode, sequence, to_date, year, month, dayofmonth, dayofweek, quarter, date_format, weekofyear, trunc, expr, when, concat, create_map, datediff, avg
from pyspark.sql.types import DateType

In [0]:
#Configuração de Schemas
catalog = "workspace_ecommerce"
silver_schema = "silver"
gold_schema = "gold"

## 1.0 Criação da Dimensão Calendário (`dim_calendario`)

Tabela essencial para Time Intelligence (YoY, MoM). Gerada via código para cobrir todos os períodos de vendas

In [0]:
# CONFIGURAÇÃO: Datas
# Ajustado para cobrir o dataset Olist (2016-2018) + margem
dt_inicio = "2015-01-01"
dt_fim = "2020-01-01"

print(f"Gerando dimensão calendário de {dt_inicio} a {dt_fim}...")

Gerando dimensão calendário de 2015-01-01 a 2020-01-01...


In [0]:
# 1. Gerar Sequencia de Datas (Equivalente ao CALENDAR que utilizo no Power BI)
df_cal = spark.sql(f"""
    SELECT explode(sequence(to_date('{dt_inicio}'), to_date('{dt_fim}'), interval 1 day)) as data_base
""")

# 2. Mapeamento Manual para Portugues
# Spark padrão retorna em Inglês, então forcarei PT-BR
mapa_meses = create_map(
    lit(1), lit('Janeiro'), lit(2), lit('Fevereiro'), lit(3), lit('Março'), lit(4), lit('Abril'), lit(5), lit('Maio'), lit(6), lit('Junho'), lit(7), lit('Julho'), lit(8), lit('Agosto'), lit(9), lit('Setembro'), lit(10), lit('Outubro'), lit(11), lit('Novembro'), lit(12), lit('Dezembro')
)

mapa_dias = create_map(
    lit(1), lit('Domingo'), lit(2), lit('Segunda-Feira'), lit(3), lit('Terça-Feira'), lit(4), lit('Quarta-Feira'), lit(5), lit('Quinta-Feira'), lit(6), lit('Sexta-Feira'), lit(7), lit('Sábado')
)

# 3. Construção das Colunas (Baseada na Linguagem DAX do Power BI)
df_dim_calendario = df_cal.select(
    col("data_base").alias("dt_referencia"),

    # "Ano"
    year("data_base").alias("nr_ano"),

    # "Mês"
    month("data_base").alias("nr_mes"),

    # "Nome_Mês"
    mapa_meses[month("data_base")].alias("desc_nome_mes"),

    # "Nome Mês Curto (3 Primeiras Letras)"
    expr("substring(element_at(map(1,'Janeiro',2,'Fevereiro',3,'Março',4,'Abril', 5, 'Maio',6,'Junho',7,'Julho',8,'Agosto',9,'Setembro',10,'Outubro',11,'Novembro',12,'Dezembro'), month(data_base)), 1, 3)").alias("desc_nome_mes_curto"),

    # "Dia"
    dayofmonth("data_base").alias("nr_dia"),

    # "Nome Dia Semana"
    mapa_dias[dayofweek("data_base")].alias("desc_dia_semana"),

    # "Nome Dia Curto"
    expr("substring(element_at(map(1, 'Dom',2,'Seg',3,'Ter',4,'Qua',5,'Qui',6,'Sex',7,'Sab'), dayofweek(data_base)), 1, 3)").alias("desc_dia_semana_curto"),

    # "Dia Semana (N°)" (DAX WEEKDAY, 2 -> Segunda=1)
    # Spark dayofweek: Dom=1, Seg=2. Para virar Seg=1, usamos lógica matemática:
    # (datofweek + 5) % 7 + 1 ajusta para Segunda=1 até Domingo=7
    ((dayofweek("data_base") + 5) % 7 + 1).alias("nr_dia_semana_iso"),

    # "Ano-Mês" (YYYY-MM)
    date_format("data_base", "yyyy-MM").alias("desc_ano_mes"),

    # "Trimestre" (T & Quarter)
    concat(lit("T"), quarter("data_base")).alias("desc_trimestre"),

    # "Ano-Trimestre" (YYYY-TQ)
    concat(year("data_base"), lit("-T"), quarter("data_base")).alias("desc_ano_trimestre"),

    # "Semana do Ano"
    weekofyear("data_base").alias("nr_semana_ano"),

    # "Início do Mês"
    trunc("data_base", "MM").alias("dt_inicio_mes"),

    # "Início da Semana" (Considerando Segunda-Feira como início)
    trunc("data_base", "week").alias("dt_inicio_semana")
)

In [0]:
# 4. Salvar na Gold
df_dim_calendario.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{gold_schema}.dim_calendario")

print("Dimensão Calendário criada com sucesso!")
display(df_dim_calendario)

Dimensão Calendário criada com sucesso!


dt_referencia,nr_ano,nr_mes,desc_nome_mes,desc_nome_mes_curto,nr_dia,desc_dia_semana,desc_dia_semana_curto,nr_dia_semana_iso,desc_ano_mes,desc_trimestre,desc_ano_trimestre,nr_semana_ano,dt_inicio_mes,dt_inicio_semana
2015-01-01,2015,1,Janeiro,Jan,1,Quinta-Feira,Qui,4,2015-01,T1,2015-T1,1,2015-01-01,2014-12-29
2015-01-02,2015,1,Janeiro,Jan,2,Sexta-Feira,Sex,5,2015-01,T1,2015-T1,1,2015-01-01,2014-12-29
2015-01-03,2015,1,Janeiro,Jan,3,Sábado,Sab,6,2015-01,T1,2015-T1,1,2015-01-01,2014-12-29
2015-01-04,2015,1,Janeiro,Jan,4,Domingo,Dom,7,2015-01,T1,2015-T1,1,2015-01-01,2014-12-29
2015-01-05,2015,1,Janeiro,Jan,5,Segunda-Feira,Seg,1,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05
2015-01-06,2015,1,Janeiro,Jan,6,Terça-Feira,Ter,2,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05
2015-01-07,2015,1,Janeiro,Jan,7,Quarta-Feira,Qua,3,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05
2015-01-08,2015,1,Janeiro,Jan,8,Quinta-Feira,Qui,4,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05
2015-01-09,2015,1,Janeiro,Jan,9,Sexta-Feira,Sex,5,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05
2015-01-10,2015,1,Janeiro,Jan,10,Sábado,Sab,6,2015-01,T1,2015-T1,2,2015-01-01,2015-01-05


## 2. Criação das Dimensões de Entidade (`dim_`)

Consolidação dos dados cadastrais e enriquecimento para análises avançadas.

* **dim_clientes:** Dados de localização dos compradores, **enriquecidos com Latitude e Longitude média** (via tabela `geolocation`) para permitir mapas de calor.
* **dim_vendedores:** Dados de localização dos sellers, também enriquecidos com coordenadas geográficas.
* **dim_produtos:** Dados físicos dos produtos + Tradução de categoria (PT/EN).
* **dim_pedidos (Bridge):** Tabela Ponte contendo o **cabeçalho do pedido** (Datas, Status, Cliente). Essencial para conectar a `fct_vendas` com a `fct_pagamentos`.

In [0]:
# PREPARAÇÃO GEO
# Agrupamos por CEP para ter uma única coordenada média por região
# Isso evita a multiplicação de linhas nos joins
df_geo_raw = spark.read.table(f"{catalog}.{silver_schema}.geolocation")
df_geo_agg = df_geo_raw.groupBy("nr_cep_prefixo") \
    .agg(
        avg("nr_latitude").alias("nr_latitude"),
        avg("nr_longitude").alias("nr_longitude")
    )

In [0]:
# DIM_CLIENTES (ENRIQUECIDA)
df_cust = spark.read.table(f"{catalog}.{silver_schema}.customers")

df_dim_clientes = df_cust.join(df_geo_agg, "nr_cep_prefixo", how="left") \
  .select(
    df_cust.id_cliente,
    df_cust.id_cliente_unico,
    df_cust.nm_cidade,
    df_cust.sg_estado,
    df_cust.nr_cep_prefixo,
    df_geo_agg.nr_latitude,
    df_geo_agg.nr_longitude
  )
df_dim_clientes.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{catalog}.{gold_schema}.dim_clientes") 

In [0]:
# DIM_VENDEDORES (ENRIQUECIDA)
df_seller = spark.read.table(f"{catalog}.{silver_schema}.sellers")

# Join com GEO
df_dim_vendedores = df_seller.join(df_geo_agg, "nr_cep_prefixo", how="left") \
    .select(
        df_seller.id_vendedor,
        df_seller.nm_cidade,
        df_seller.sg_estado,
        df_seller.nr_cep_prefixo,
        df_geo_agg.nr_latitude,
        df_geo_agg.nr_longitude
    )
df_dim_vendedores.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{catalog}.{gold_schema}.dim_vendedores")

In [0]:
# DIM_PRODUTOS
df_prod = spark.read.table(f"{catalog}.{silver_schema}.products")
df_trad = spark.read.table(f"{catalog}.{silver_schema}.product_category_name_translation")

In [0]:
# DIM_PRODUTOS
df_dim_produtos = df_prod.join(df_trad, on=df_prod.desc_categoria == df_trad.desc_categoria_pt, how="left") \
    .select(
        df_prod.id_produto,
        df_prod.desc_categoria.alias("desc_categoria_pt"),
        df_trad.desc_categoria_en, # Coluna Traduzida
        df_prod.nr_comprimento_cm,
        df_prod.nr_largura_cm,
        df_prod.nr_altura_cm,
        df_prod.nr_peso_gramas,
        df_prod.nr_qtd_fotos
    )

df_dim_produtos.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{catalog}.{gold_schema}.dim_produtos")

In [0]:
# DIM_PEDIDOS (A BRIDGE/PONTE)
# Esta tabela terá apenas 1 linha por pedido. Ela conecta as duas Fatos.
df_orders = spark.read.table(f"{catalog}.{silver_schema}.orders")
df_dim_pedidos = df_orders.select(
    "id_pedido",
    "id_cliente", # Para ligar cliente ao pedido
    "desc_situacao",
    "dt_compra", # Data principal para ligar ao calendário
    "dt_entrega_cliente",
    "dt_estimada_entrega"
).distinct()

df_dim_pedidos.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{gold_schema}.dim_pedidos")

print("Dimensões criadas com sucesso!")

Dimensões criadas com sucesso!


## 3. Criação das Tabelas Fato (`fct_`)

Separamos os fatos em duas tabelas para respeitar a granularidade dos dados e evitar erros de soma (Fan Trap)

1. **`fct_vendas`**: Foco em Logística, Receita de Produtos e Qualidade (Reviews)
  * *Granularidade*: Item do Pedido.
  * *Enriquecimento*: Traz o sentimento calculado pela IA ('reviews_analise_ia').

2. **`fct_pagamentos`**: Foco Financeiro.
  * *Granularidade:* Parcela/Método de Pagamento.

In [0]:
print("Processando Tabelas Fato...")

# Carregando tabelas Silver necessárias
df_orders = spark.read.table(f"{catalog}.{silver_schema}.orders")
df_items = spark.read.table(f"{catalog}.{silver_schema}.order_items")
df_reviews = spark.read.table(f"{catalog}.{silver_schema}.reviews_analise_ia") # Tabela com Analise da IA
df_payments = spark.read.table(f"{catalog}.{silver_schema}.order_payments")

Processando Tabelas Fato...


In [0]:
# 3.1 FATO VENDAS
# JOIN: items (base) + Orders (Datas/Status) + Reviews (Qualidade)
df_fct = df_items.join(df_orders, on="id_pedido", how="inner")
df_fct_full = df_fct.join(df_reviews, on="id_pedido", how="left")

df_fct_vendas = df_fct_full.select(
    # CHAVES (FKs)
    col("id_pedido"),
    col("id_cliente"),
    col("id_produto"),
    col("id_vendedor"),

    # Datas (Para Time Intelligence)
    col("dt_compra"),
    col("dt_aprovacao"),
    col("dt_entrega_cliente"),
    col("dt_estimada_entrega"),

    # Métricas Financeiras
    col("vl_preco"),
    col("vl_frete"),
    (col("vl_preco") + col("vl_frete")).alias("vl_total_item"),

    # Métricas de Performance (Lead Time em Dias)
    datediff(col("dt_entrega_cliente"), col("dt_compra")).alias("qtd_dias_entrega_real"),
    datediff(col("dt_estimada_entrega"), col("dt_entrega_cliente")).alias("qtd_dias_diferenca_estimado"),

    # Atributos Qualitativos
    col("desc_situacao"),
    col("vl_nota").alias("nr_nota_review"),
    col("sentimento_ia") # Resultado Forjado
)

df_fct_vendas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog}.{gold_schema}.fct_vendas")

In [0]:
# 3.2 FATO PAGAMENTOS
# JOIN: Payments + Orders (Apenas para pegar a data da compra)
df_orders_lite = df_orders.select("id_pedido", "dt_compra")

df_fct_pagamentos = df_payments.join(df_orders_lite, "id_pedido", "inner") \
    .select(
        col("id_pedido"),
        col("dt_compra"), #Necessário para ligar na dim_calendario
        col("desc_tipo_pagamento"),
        col("nr_parcelas"),
        col("vl_pagamento")
    )

df_fct_pagamentos.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{gold_schema}.fct_pagamentos")

print("Fato Vendas e Fato Pagamentos criadas com sucesso!")

Fato Vendas e Fato Pagamentos criadas com sucesso!


## 4. Validação e Analytics

Verificando os dados finais com uma consulta SQL simples para garantir que os joins funcionaram

In [0]:
%sql
-- Validação Visual: Top 5 categorias de vendas com Sentimento IA
SELECT
  p.desc_categoria_pt,
  v.sentimento_ia,
  count(*) as total_vendas
FROM workspace_ecommerce.gold.fct_vendas v
LEFT JOIN workspace_ecommerce.gold.dim_produtos p ON v.id_produto = p.id_produto
WHERE v.sentimento_ia IS NOT NULL
GROUP BY p.desc_categoria_pt, v.sentimento_ia
ORDER BY total_vendas DESC
LIMIT 10;

desc_categoria_pt,sentimento_ia,total_vendas
utilidades_domesticas,Neutro,15
beleza_saude,Neutro,9
cama_mesa_banho,Positivo,7
moveis_decoracao,Neutro,7
relogios_presentes,Neutro,7
esporte_lazer,Neutro,6
construcao_ferramentas_seguranca,Neutro,6
informatica_acessorios,Neutro,5
cama_mesa_banho,Neutro,5
papelaria,Neutro,5


----------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
%sql
SELECT
  v.desc_situacao,
  v.sentimento_ia,
  count(*) as total_vendas
FROM workspace_ecommerce.gold.fct_vendas v
WHERE v.sentimento_ia IS NOT NULL
GROUP BY v.desc_situacao, v.sentimento_ia
ORDER BY total_vendas DESC
LIMIT 10;

desc_situacao,sentimento_ia,total_vendas
delivered,Neutro,97
delivered,Positivo,27
delivered,Negativo,9
canceled,Neutro,2
processing,Negativo,2
canceled,Negativo,1
shipped,Neutro,1
